In [4]:
import pandas as pd


# Limpieza tablas extraidas desde la API PLACE de Google: 

#### Todas las tablas tienen la misma estructura por lo que la misma limpieza se la aplico a todas.

In [5]:
caribe = pd.read_csv('../data/caribe.csv', encoding='latin1')
coreano = pd.read_csv('../data/coreano.csv', encoding='latin1')
india = pd.read_csv('../data/india.csv', encoding='latin1')
ita = pd.read_csv('../data/ita.csv', encoding='latin1')
japo = pd.read_csv('../data/japo.csv', encoding='latin1')
mex = pd.read_csv('../data/mex.csv', encoding='latin1')
peru = pd.read_csv('../data/peru.csv', encoding='latin1')
chino = pd.read_csv('../data/chino.csv', encoding='latin1')

In [6]:
coreano.info() # Aqui veo que solo price-level tiene nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   business_status     20 non-null     object 
 1   geometry            20 non-null     object 
 2   name                20 non-null     object 
 3   photos              20 non-null     object 
 4   price_level         12 non-null     float64
 5   rating              20 non-null     float64
 6   types               20 non-null     object 
 7   user_ratings_total  20 non-null     int64  
 8   tipo                20 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.5+ KB


In [7]:
coreano.columns

Index(['business_status', 'geometry', 'name', 'photos', 'price_level',
       'rating', 'types', 'user_ratings_total', 'tipo'],
      dtype='object')

In [8]:
coreano=coreano.drop(columns=['business_status','geometry','tipo'])

In [9]:
coreano.columns = ['Nombre','Foto','Precio','Puntuacion','Servicios', 'Nº opiniones']

In [10]:
coreano.Nombre = coreano.Nombre.str.upper()

In [11]:
coreano.head() 

,Nombre,Foto,Precio,Puntuacion,Servicios,Nº opiniones
0,RESTAURANTE COREANO GANGNAM,"[{'height': 2988, 'html_attributions': ['<a hr...",1.0,4.4,"['restaurant', 'food', 'point_of_interest', 'e...",597
1,RESTAURANTE KOREA,"[{'height': 2773, 'html_attributions': ['<a hr...",2.0,4.5,"['restaurant', 'food', 'point_of_interest', 'e...",1922
2,GO HYANG MAT,"[{'height': 3000, 'html_attributions': ['<a hr...",1.0,4.5,"['restaurant', 'food', 'point_of_interest', 'e...",994
3,RESTAURANTE MIDANG,"[{'height': 1669, 'html_attributions': ['<a hr...",2.0,4.7,"['meal_delivery', 'restaurant', 'food', 'point...",762
4,RESTAURANTE MARU,"[{'height': 3024, 'html_attributions': ['<a hr...",2.0,4.3,"['restaurant', 'food', 'point_of_interest', 'e...",848


#### Creo una función para limpiar 🫧🧹 todo de una.

In [12]:
import sys
sys.path.append("..")
from src.funciones import *  #funciones

In [13]:
india = clean(india) 
ita = clean(ita) 
japo = clean(japo)
mex = clean(mex) 
peru = clean(peru)
chino = clean(chino)

In [14]:
peru

,Nombre,Foto,Precio,Puntuacion,Servicios,Nº opiniones
0,PISCOMAR RESTAURANTE PERUANO BY JHOSEF ARIAS,"[{'height': 4000, 'html_attributions': ['<a hr...",2.0,4.6,"['restaurant', 'food', 'point_of_interest', 'e...",4353
1,PE DE PISCO RESTAURANTE PERUANO,"[{'height': 3024, 'html_attributions': ['<a hr...",1.0,4.0,"['bar', 'restaurant', 'food', 'point_of_intere...",1031
2,CHINCHA CASA PERÃº,"[{'height': 2592, 'html_attributions': ['<a hr...",2.0,4.4,"['restaurant', 'food', 'point_of_interest', 'e...",1399
3,PISCO 41 RESTAURANTE PERUANO,"[{'height': 3000, 'html_attributions': ['<a hr...",2.0,4.2,"['restaurant', 'food', 'point_of_interest', 'e...",992
4,RESTAURANTE HUACATAY,"[{'height': 1192, 'html_attributions': ['<a hr...",1.0,4.0,"['restaurant', 'food', 'point_of_interest', 'e...",573
5,RESTAURANTE PERUANO SHARI'S,"[{'height': 3024, 'html_attributions': ['<a hr...",1.0,3.8,"['restaurant', 'food', 'point_of_interest', 'e...",435
6,RESTAURANTE MIS TRADICIONES,"[{'height': 3000, 'html_attributions': ['<a hr...",2.0,4.2,"['restaurant', 'food', 'point_of_interest', 'e...",2558
7,RESTAURANTE MIKUNA,"[{'height': 3024, 'html_attributions': ['<a hr...",NaN,4.5,"['bar', 'restaurant', 'food', 'point_of_intere...",745
8,INTI DE ORO BARRIO DE LAS LETRAS,"[{'height': 2624, 'html_attributions': ['<a hr...",2.0,4.5,"['bar', 'restaurant', 'food', 'point_of_intere...",1930
9,LA BUENA MESA PERÃº,"[{'height': 3024, 'html_attributions': ['<a hr...",1.0,4.4,"['restaurant', 'food', 'point_of_interest', 'e...",483


#### La talba de Caribe tiene una columna menos por lo que la funcion no se puede aplicar

In [15]:
caribe=caribe.drop(columns=['business_status','geometry','tipo'])
caribe.columns = ['Nombre','Foto','Puntuacion','Servicios', 'Nº opiniones']
caribe.Nombre = caribe.Nombre.str.upper()

#### Nuesto principal objetivo 🎯🎯 es tener todos los nombres de los restaurantes de las tablas para poder relacionarlos con la tabla de locales

In [16]:
locales = pd.read_csv('../data/restaurantes.csv', encoding='latin1')

In [17]:
locales.columns = ['o', 'id_local','Nombre']

In [18]:
df_list = [caribe, japo, chino, mex, peru, india, ita]


results = []

for df in df_list:
    result = locales.merge(df, on='Nombre', how='inner')
    results.append(result)

# Concatenar todos los resultados en un solo DataFrame
result = pd.concat(results, ignore_index=True)

In [19]:
result = result.drop_duplicates(subset=['id_local'], keep=False)


In [20]:
result= result.drop(columns=['o','Foto'])

In [21]:
result.head()

,id_local,Nombre,Puntuacion,Servicios,Nº opiniones,Precio
0,17202,KABUKI,4.6,"['restaurant', 'food', 'point_of_interest', 'e...",876.0,4.0
1,50673,KABUKI,4.6,"['restaurant', 'food', 'point_of_interest', 'e...",876.0,4.0
2,33122,HATTORI HANZO,3.9,"['restaurant', 'food', 'point_of_interest', 'e...",3484.0,2.0
3,78750,PILAR AKANEYA,4.8,"['restaurant', 'food', 'point_of_interest', 'e...",1189.0,NaN
4,1792,RESTAURANTE CHINO GRAN SIGLO,4.3,"['restaurant', 'food', 'point_of_interest', 'e...",636.0,1.0


In [22]:
result.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 0 to 54
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_local      25 non-null     int64  
 1   Nombre        25 non-null     object 
 2   Puntuacion    25 non-null     object 
 3   Servicios     25 non-null     object 
 4   Nº opiniones  23 non-null     float64
 5   Precio        24 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 6.2 KB


In [23]:
optim(result)
result.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 0 to 54
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   id_local      25 non-null     int32   
 1   Nombre        25 non-null     category
 2   Puntuacion    25 non-null     category
 3   Servicios     25 non-null     category
 4   Nº opiniones  23 non-null     float32 
 5   Precio        24 non-null     float32 
dtypes: category(3), float32(2), int32(1)
memory usage: 4.7 KB


In [24]:
result.to_csv('../data/Opiniones.csv',index=False)

In [25]:
india = optim(india) 
ita = optim(ita) 
japo = optim(japo)
mex = optim(mex) 
peru = optim(peru)
chino = optim(chino)
caribe = optim(caribe)

In [26]:
india.to_json('../data/Indio.json',index=False)
ita.to_json('../data/Italiano.json',index=False)
japo.to_json('../data/Japonés.json',index=False)
mex.to_json('../data/Mexicano.json',index=False)
peru.to_json('../data/Peruano.json',index=False)
chino.to_json('../data/Chino.json',index=False)
caribe.to_json('../data/Caribe.json',index=False)